In [1]:
%cd drive/MyDrive/pythonlibs

/content/drive/MyDrive/pythonlibs


In [2]:
import numpy as np
import os
import pandas as pd

In [3]:
"""
 - NHANES data from 2009-2018 (biannual survey)
 - contains demographics, diet, lab test values, clinical examinations, 
 and questionairres
 - scopes of data vary within surveys -> will only take common denominators
"""
dir = "../nhanes_0918/"

year_list =["F", "G", "H", "I", "J"]
pd_dict = {}
for year in year_list:
    path = dir + year
    pd_dict[year] = []
    for file in os.listdir(path):
        temp_dir = path + "/" + str(file)
        #print(temp_dir)
        try:
            temp_pd = pd.read_sas(temp_dir)
            pd_dict[year].append(temp_pd)
        except:
            continue


/usr/local/lib/python3.7/dist-packages/pandas/io/sas/sas_xport.py:475: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df[x] = v


In [4]:
df_temp = pd.DataFrame()
df_noid = pd.DataFrame()
df_list = []
final_df = pd.DataFrame()
for year, dfs in pd_dict.items():
    for i, df in enumerate(dfs):
        if i == 0:
            df_temp = df
        else:
            try:
                df_noid = df.drop(['SEQN'], axis=1, inplace=False)
            except: # dataframe out of interest
                continue
            df_temp = pd.concat([df_temp, df_noid], axis=1, join='inner')
            cols = [c for c in df_temp.columns if c[-2:] !='SI']
            df_temp = df_temp[cols]
            df_temp = df_temp.loc[:,~df_temp.columns.duplicated()]
            
    df_list.append(df_temp)
# merge dfs 
for i, df in enumerate(df_list):
    if i == 0:
        final_df = df
    else:
        common_cols = np.intersect1d(final_df.columns, df.columns)
        for col in final_df.columns:
            if col not in common_cols:
                try:
                    final_df = final_df.drop(col, axis=1, inplace=False)
                except:
                    continue
        for col in df.columns:
            if col not in common_cols:
                try:
                    df = df.drop(col, axis=1, inplace=False)
                except:
                    continue
        final_df = pd.concat([final_df, df])
# defining label(y)
final_df = final_df[final_df['OHAREC'].notnull()] # remove rows w/ no labels
y = (final_df['OHAREC'] <=3).astype(float)
final_df = final_df.drop(['OHAREC'], 
                         axis=1, inplace=False) # remove label from data
final_df = final_df.drop(["DSDSUPP", "SMDUPCA", "SMD100BR",
                          "RXDDRUG", "RXDDRGID"]
                         , axis=1, inplace=False) # columns with text data

#drop cols with percentage of null values > 70%
perc = 70.0 
min_count =  int(((100-perc)/100)*final_df.shape[0] + 1)
final_df = final_df.dropna(axis=1, thresh=min_count)
delete_cols = ['URDUCSLC', 'URDUMOLC']
for col in delete_cols:
    final_df = final_df.drop([col], axis=1, inplace=False)
nan_col_list = final_df.columns[final_df.isna().any()].tolist()
#categorical_list = ['WTDRD1', 'WTDR2D','LBDHDD', 'LBXTC', 'BPXDI2', 'BPXSY2', 'BPXSY1', 'BPXML1', 'BPXPLS', 'BPD035', 'LBXSTR']

subs = 'LC'
  
# using list comprehension 
# to get string with substring 
lc_list = [i for i in final_df.columns if subs in i]  # list of cols to remove
lc_list.remove('DR1TALCO')
lc_list.remove('DR2TALCO')
lc_list.remove('DR2IALCO')

for col in nan_col_list:
    
    if final_df[col].mode()[0] == np.nan or col in lc_list:
        final_df = final_df.drop([col], axis=1, inplace=False)
        print(col)
        continue
    if (final_df[col].min() == int(final_df[col].min())) and (final_df[col].max() == int(final_df[col].max())) and (final_df[col].mode()[0] == int(final_df[col].mode()[0])): # high probability of being categorical data
        #final_df = final_df.fillna(value={col : final_df[col].mode()[0]})
        dummies = pd.get_dummies(final_df[col], prefix=col)
        if dummies.shape[1] > 25: 
            final_df = final_df.fillna(value={col : final_df[col].mean()})
            continue
        final_df[col] = final_df[col].fillna(final_df[col].mode()[0])
        final_df = final_df.drop([col], axis=1, inplace=False)
        final_df = pd.concat([final_df, dummies], axis=1, join='outer')
    
    else: # high probability of being numerical data
        final_df = final_df.fillna(value={col : final_df[col].mean()})

categorical_features = []
numerical_features = []
for col in final_df.columns:
    if (final_df[col].min() == int(final_df[col].min())) and (final_df[col].max() == int(final_df[col].max())) and (final_df[col].mode()[0] == int(final_df[col].mode()[0])):
        categorical_features.append(col)
    else:
        numerical_features.append(col)
for col in numerical_features:
    final_df[col] = (final_df[col] - final_df[col].mean()) / final_df[col].std()          
final_df_no_seqn = final_df.drop(['SEQN'], axis=1, inplace=False)
X_ = final_df_no_seqn.to_numpy()  
y_ = y.to_numpy()
final_df['WTMEC2YR'] = 1/5 * final_df['WTMEC2YR']
X_.tofile('../nhanes_data/data_onehot_encoded_final.dat') # dimension of X_ is (-1, 742). Load as follows
"""
X = np.fromfile('../nhanes_data/data.dat').reshape(-1,742)
"""
y_.tofile('../nhanes_data/label.dat')
final_df.to_pickle('../nhanes_data/final_df.pkl')

DR1TCALC
DR2TCALC
DR2ICALC
DS1ICALC
DS2ICALC
DSQICALC
LBDBCDLC
LBDBPBLC
LBDTHGLC
URDCNPLC
URDCOPLC
URDECPLC
URDMBPLC
URDMC1LC
URDMEPLC
URDMHHLC
URDMHPLC
URDMNPLC
URDMOHLC
URDMZPLC
URDMIBLC
URDUA3LC
URDUA5LC
URDUABLC
URDUACLC
URDUDALC
URDUBALC
URDUCDLC
URDUCOLC
URDUPBLC
URDUSBLC
URDUTLLC
URDUTULC


In [5]:
len(numerical_features), len(categorical_features)

(375, 1162)

In [6]:
final_df.columns[final_df.isna().any()].tolist()

[]

In [ ]:
categorical_features

['SEQN',
 'SDDSRVYR',
 'RIDSTATR',
 'RIAGENDR',
 'RIDRETH1',
 'DMDHHSIZ',
 'DMDFMSIZ',
 'DMDHRGND',
 'SIALANG',
 'SIAINTRP',
 'SDMVPSU',
 'SDMVSTRA',
 'DR1DRSTZ',
 'DR1EXMER',
 'DR2DRSTZ',
 'DR2ILINE',
 'DR2_030Z',
 'DR2IFDCD',
 'DS1LOC',
 'DS1MTCH',
 'DS2LOC',
 'DS2MTCH',
 'DSDMTCH',
 'DSD090',
 'DSD103',
 'DSD010',
 'DSD010AN',
 'OHDEXSTS',
 'OHDDESTS',
 'OHX01TC',
 'OHX02TC',
 'OHX03TC',
 'OHX04TC',
 'OHX05TC',
 'OHX06TC',
 'OHX07TC',
 'OHX08TC',
 'OHX09TC',
 'OHX10TC',
 'OHX11TC',
 'OHX12TC',
 'OHX13TC',
 'OHX14TC',
 'OHX15TC',
 'OHX16TC',
 'OHX17TC',
 'OHX18TC',
 'OHX19TC',
 'OHX20TC',
 'OHX21TC',
 'OHX22TC',
 'OHX23TC',
 'OHX24TC',
 'OHX25TC',
 'OHX26TC',
 'OHX27TC',
 'OHX28TC',
 'OHX29TC',
 'OHX30TC',
 'OHX31TC',
 'OHX32TC',
 'HSAQUEX',
 'HIQ011',
 'RXDUSE',
 'SMAQUEX2',
 'SMAQUEX',
 'WHD010',
 'WHD020',
 'WHQ030',
 'WHQ040',
 'WHD050',
 'LBXSBU',
 'BMDSTATS',
 'BPQ020',
 'ECD010',
 'HUQ010',
 'HUQ030',
 'HUQ071',
 'IMQ020',
 'OHDRCSTS',
 'SLQ050',
 'RIDEXMON_1.0',
 'RIDEXMON_2.